In [51]:
import pandas as pd
from json import loads
from kafka import KafkaConsumer
from time import sleep
import json
import configparser

In [57]:
config = configparser.ConfigParser()
dev_config = config.read('dev')

In [ ]:
consumer = KafkaConsumer(dev_config['TOPIC'], 
    bootstrap_servers=[f"{dev_config['BOOTSTRAP_SERVER']}:9092"],
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)

In [60]:
consumer = KafkaConsumer('kafka-stock-market-topic', 
    bootstrap_servers=['54.176.97.117:9092'],
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)


NoBrokersAvailable: NoBrokersAvailable

In [31]:
from boto3 import Session
session = Session(profile_name='raghav_personal')
s3 = session.client('s3')
bucket_name = "kafka-stock-market-project-raghav"


In [47]:
# find latest file number in case the consumer stops - should stop file from overwriting : based on file name 
# real life = would be on timestamp   

paginator = s3.get_paginator("list_objects_v2")
page_iterator = paginator.paginate(Bucket=bucket_name, Prefix='stock_market')

latest = None
for page in page_iterator:
    if "Contents" in page:
        latest2 = max(page['Contents'], key=lambda x: int(x['Key'][:-5].split("_")[2]))
        if latest is None or latest2['Key'] > latest['Key']:
            latest = latest2

count = int(latest['Key'][:-5].split("_")[2])+1



In [49]:
from io import BytesIO
for i in consumer:
    # convert json.dumps formed string to bytes using bytesIO
    file_object = BytesIO(json.dumps(i.value).encode('utf-8'))
    # upload file object to s3 bucket 
    s3.upload_fileobj(file_object, bucket_name, f"stock_market_{count}.json")
    count += 1

KeyboardInterrupt: 